In [66]:
from datetime import datetime, date, timezone
import pandas as pd
import requests

In [47]:
DATE_FROM = "2022-05-01"
DATE_TO = "2022-06-01"

YAKSWAP_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/yieldyak/yak-aggregator"

WAVAX = "0xB31f66AA3C1e785363F0875A1B74E27b85FD66c7".lower()
USDC = "0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e".lower()
USDCe = "0xA7D7079b0FEaD91F3e65f86E8915Cb59c1a4C664".lower()
YYAVAX = "0xF7D9281e8e363584973F946201b82ba72C965D27".lower()
DAI = "0xd586E7F844cEa2F87f50152665BCbc2C279D8d70".lower()
USDT = "0x9702230A8Ea53601f5cD2dc00fDBc13d4dF4A8c7".lower()

## Fetch & Parse data

In [67]:
def date_str_to_timestamp(_date_str_utc):
    date_object = datetime \
        .strptime(_date_str_utc, "%Y-%m-%d") \
        .replace(tzinfo=timezone.utc)
    return int(date_object.timestamp())

def make_graph_query_request(_query):
    return requests.post(YAKSWAP_GRAPH_URL, json={'query': _query}).json()

def get_swaps(timestamp_from, timestamp_to):
    full_result = []
    while 1:
        print(f"querying between {timestamp_from}-{timestamp_to}")
        query = """
        {
            swapTransactions(
                orderBy: blockNumber,
                orderDirection: asc,
                where: { blockTimestamp_gte: %d, blockTimestamp_lte: %d },
                first: 1000
            ) {
                id
                blockNumber
                blockTimestamp
                swaps {
                    trader { id }
                }
                underlyingSwaps {
                    fromAmount
                    toAmount
                    fromToken { id, symbol, decimals }
                    toToken { id, symbol, decimals }
                    adapter { name }
                    logIndex
                }
            }
        }
        """ % (timestamp_from, timestamp_to)
        result = make_graph_query_request(query)["data"]["swapTransactions"]
        full_result += result
        print('found', len(result), "results")
        if len(result) < 1000:
            return full_result
        # Set timestamp gt than the most recent result
        # Note: this will skip any remaning tx in the block
        timestamp_from = int(result[-1]["blockTimestamp"])

In [69]:
tm_from = date_str_to_timestamp(DATE_FROM)
tm_to = date_str_to_timestamp(DATE_TO)
swaps_raw = get_swaps(tm_from, tm_to)

querying between 1651363200-1654041600
found 1000 results
querying between 1651500953-1654041600
found 1000 results
querying between 1651661905-1654041600
found 1000 results
querying between 1651822531-1654041600
found 1000 results
querying between 1651978256-1654041600
found 1000 results
querying between 1652094108-1654041600
found 1000 results
querying between 1652142333-1654041600
found 1000 results
querying between 1652208977-1654041600
found 1000 results
querying between 1652252981-1654041600
found 1000 results
querying between 1652277987-1654041600
found 1000 results
querying between 1652308071-1654041600
found 1000 results
querying between 1652341827-1654041600
found 1000 results
querying between 1652384448-1654041600
found 1000 results
querying between 1652464185-1654041600
found 1000 results
querying between 1652527878-1654041600
found 1000 results
querying between 1652591330-1654041600
found 1000 results
querying between 1652693931-1654041600
found 1000 results
querying betwe

In [70]:
normalize = lambda a, d: int(a) / 10**int(d)
adapter_swaps = []

for swap_tx in swaps_raw:
    swaps = []
    blockTimestamp = pd.to_datetime(swap_tx['blockTimestamp'], unit='s')
    trader = swap_tx["swaps"][0]["trader"]["id"] if swap_tx["swaps"] else "unknown"
    blockNumber = int(swap_tx["blockNumber"])
    txHash = swap_tx["id"]
    for swap in swap_tx["underlyingSwaps"]:
        adapter_swaps.append({
            "fromAmount": normalize(swap["fromAmount"], swap["fromToken"]["decimals"]),
            "toAmount": normalize(swap["toAmount"], swap["toToken"]["decimals"]),
            "fromToken": swap["fromToken"]["id"],
            "fromTokenName": swap["fromToken"]["id"],
            "toToken": swap["toToken"]["id"],
            "toTokenName": swap["toToken"]["id"],
            "adapter": swap["adapter"]["name"],
            "blockTimestamp": blockTimestamp,
            "blockNumber": blockNumber,
            "trader": trader,
            "txHash": txHash,
            "logIndex": int(swap["logIndex"])
        })

adapter_swaps_df = pd.DataFrame(adapter_swaps)

/var/folders/8t/96s8_s3958l2jm63574ccr200000gn/T/ipykernel_74239/2553805351.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  blockTimestamp = pd.to_datetime(swap_tx['blockTimestamp'], unit='s')


In [71]:
adapter_swaps_sorted = adapter_swaps_df.sort_values(by=['txHash', 'logIndex'], ascending=True)
adapter_swaps = adapter_swaps_sorted.groupby(['txHash', 'trader']).agg(
    blockNumber=('blockNumber', 'first'),
    blockTimestamp=('blockTimestamp', 'first'),
    adapters=('adapter', lambda x: list(x.unique())),
    fromAmount=('fromAmount', 'first'),
    toAmount=('toAmount', 'last'),
    fromToken=('fromToken', 'first'),
    toToken=('toToken', 'last')
).reset_index()

adapter_swaps['tokens'] = adapter_swaps_sorted.groupby(['txHash', 'trader']).apply(
    lambda x: list(x[['fromToken']].values.ravel()) + [x['toToken'].iloc[-1]]
).reset_index(drop=True)


/var/folders/8t/96s8_s3958l2jm63574ccr200000gn/T/ipykernel_74239/2357458030.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  adapter_swaps['tokens'] = adapter_swaps_sorted.groupby(['txHash', 'trader']).apply(


In [75]:
STABLECOINS = [ USDC, USDCe, DAI, USDT ]
BASE_TOKEN = [ WAVAX, YYAVAX ] + STABLECOINS

COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/"
TKN_TO_COINGECKO_ID = {}
TKN_TO_COINGECKO_ID[WAVAX] = "wrapped-avax"
TKN_TO_COINGECKO_ID[YYAVAX] = "yield-yak-avax"
DAYS_AGO = (date.today() - datetime.fromtimestamp(tm_from).date()).days

def get_tkn_prices(tkn_label):
    tkn_id = TKN_TO_COINGECKO_ID.get(tkn_label)
    if not tkn_id:
        raise Exception(f"Token label: '{tkn_label}' not supported!")
    full_url = COINGECKO_URL + tkn_id + "/market_chart"
    res = requests.get(full_url, {
        "vs_currency": "usd",
        "interval": "daily",
        "days": (DAYS_AGO + 1),
    })

    return res.json()["prices"]

prices = {}
# Fetch prices for base tokens that are not pegged to USD
for tkn in [_tkn for _tkn in BASE_TOKEN if _tkn not in STABLECOINS]:
    try:
        tkn_prices = get_tkn_prices(tkn)
        tkn_prices_df = pd.DataFrame(tkn_prices, columns=["date", "price"])
        tkn_prices_df["date"] = pd.to_datetime(tkn_prices_df["date"], unit='ms').dt.date
        tkn_prices_df.set_index("date", drop=True, inplace=True)
        # Save without duplicates
        prices[tkn] = tkn_prices_df.loc[~tkn_prices_df.index.duplicated()]
    except Exception as e:
        print(f"Error raised while fetching prices for token {tkn}", str(e))

# Return token price for a particular date
# For stablecoins always return price of 1
def get_tkn_price_for_date(date, tkn):
    return 1 if tkn in STABLECOINS else float(prices[tkn].loc[date])

def get_profit_val_usd(row):
    # Return null if neither token is base-token
    if row["fromToken"] in BASE_TOKEN:
        price = get_tkn_price_for_date(
            row["blockTimestamp"].date(),
            row["fromToken"]
        )
        return row["profit"] * price

## Arbs

In [73]:
adapter_swaps["blockTimestamp"].max(), adapter_swaps["blockTimestamp"].min()

(Timestamp('2022-05-31 23:57:52'), Timestamp('2022-05-01 00:00:00'))

In [76]:
yakswap_arbs = adapter_swaps.loc[lambda df: df["fromToken"] == df["toToken"]].copy()
yakswap_arbs.loc[:, "profit"] = yakswap_arbs["toAmount"] - yakswap_arbs["fromAmount"]
yakswap_arbs.loc[:, "profit_usd"] = yakswap_arbs.apply(get_profit_val_usd, axis=1)

/var/folders/8t/96s8_s3958l2jm63574ccr200000gn/T/ipykernel_74239/140386026.py:39: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return 1 if tkn in STABLECOINS else float(prices[tkn].loc[date])


### Most profitable

In [79]:
yakswap_arbs \
    [["txHash", "blockNumber", "trader", "fromToken", "fromAmount", "profit_usd", "profit", "adapters"]] \
    .sort_values(by="profit_usd", ascending=False) \
    .head(10)

,txHash,blockNumber,trader,fromToken,fromAmount,profit_usd,profit,adapters
13486,0x83bf4f984fdb9ba3446696ea03c8f3f1cc59ac68c5c9...,14489672,0x7b3dedf3b349b8a943c18372972d150a572d2018,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e,100000.0,3960.550435,3960.550435,"[Curve3poolfAdapterV0, PlatypusYakAdapterV2]"
4750,0x2e39774f8d458623022d670c486fad761ad8223cd519...,14544255,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,900.0,3646.729995,81.907471,"[TraderJoeYakAdapterV0, Curve3poolfAdapterV0]"
19458,0xbf10c5444e6dc1d3cdc17d21329babf1f1645ae53456...,14544255,0x7b3dedf3b349b8a943c18372972d150a572d2018,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7,25000.0,3563.812719,3563.812719,"[PlatypusYakAdapterV2, Curve3poolfAdapterV0]"
12620,0x7babf41f9677921294c000446e4aaad1c890a5c314be...,14546799,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,2965.834380,66.614198,"[TraderJoeYakAdapterV0, Curve3poolfAdapterV0]"
1331,0x0ca0bac80b723c3cc8b26746b598d891c29308c71911...,14546333,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1000.0,2508.429801,56.340651,"[GmxAdapterV0, Curve3poolfAdapterV0, TraderJoe..."
14088,0x899870881cf190b5a923bcec3de7e844c0d5b1fe6b62...,14555049,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1350.0,2092.323495,46.994685,"[TraderJoeYakAdapterV0, GmxAdapterV0]"
890,0x082ecfb5c8e95d382c7101fd54c0b6a3706e81665ffb...,14544681,0x7b3dedf3b349b8a943c18372972d150a572d2018,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7,25000.0,1883.749191,1883.749191,"[CurveYUSDAdapter, Curve3poolfAdapterV0]"
4293,0x29a27dc27894e8d10ec959ab6308701e053a3a293acb...,14574968,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,2000.0,1751.819998,39.346797,"[GmxAdapterV0, CurveAtricryptoAdapterV0]"
23953,0xeb0cd53c82d7e7e4ba992bff24e7021e0f9c0c727ee9...,14548069,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1250.0,1665.208779,37.401464,"[TraderJoeYakAdapterV0, Curve3poolfAdapterV0]"
2482,0x1847099e39553782e2a143c144eedfeb1e12d806c634...,14554874,0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,1350.0,1473.044587,33.085355,"[TraderJoeYakAdapterV0, GmxAdapterV0]"


## Stats

### General

In [95]:
arb_count = yakswap_arbs.shape[0]
total_profit_usd = yakswap_arbs["profit_usd"].sum()
profit_usd_wavax = yakswap_arbs.loc[lambda df: df["fromToken"] == WAVAX]["profit_usd"].sum()
profit_usd_usdc = yakswap_arbs.loc[lambda df: df["fromToken"] == USDC]["profit_usd"].sum()
arbitrageur_count = yakswap_arbs["trader"].nunique()

print(f"Total arbitrage opportunities: {arb_count}")
print(f"Total profit: ${total_profit_usd:.2f}")
print(f"Profit with WAVAX as base: ${profit_usd_wavax:.2f}")
print(f"Profit with USDC as base: ${profit_usd_usdc:.2f}")
print(f"Total arbitrageurs: {arbitrageur_count}")

Total arbitrage opportunities: 12765
Total profit: $236550.56
Profit with WAVAX as base: $148735.12
Profit with USDC as base: $58780.41
Total arbitrageurs: 138


### By traders

In [84]:
yakswap_arbs \
    .groupby(by=["trader", "fromToken"]).agg(
        profit_usd_sum=("profit_usd", "sum"),
        profit_usd_mean=("profit_usd", "mean"),
        count=("txHash", "count")
    ) \
    .sort_values(by="profit_usd_sum", ascending=False) \
    .head(10)

profit_usd_sum  \
trader                                     fromToken                                                    
0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7   137343.811576   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e    52985.858676   
                                           0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664    19750.208872   
                                           0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7     9259.370794   
0xa85a4eef551e083b2350a20994fd5816583cae3b 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e     5710.348617   
                                           0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7     4086.624727   
0xa7b43b7afd3309fe93bb9f17145ce97ccda4f65e 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7     2970.863697   
0x6e74b6274e242f54840f78c8fbabae98b336286b 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7     1184.246601   
0xe095efc08fa11ab0891e0682a3a71ccf972eb6f7 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7      881.570059   
0xe0527e66ed8c7e372007de0d65a5eb45cac53e07 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7      793.923958   

                                                                                       profit_usd_mean  \
trader                                     fromToken                                                     
0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7        25.938397   
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e        43.148093   
                                           0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664        36.238915   
                                           0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7        40.611275   
0xa85a4eef551e083b2350a20994fd5816583cae3b 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e         3.806899   
                                           0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7         3.454459   
0xa7b43b7afd3309fe93bb9f17145ce97ccda4f65e 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7         3.955877   
0x6e74b6274e242f54840f78c8fbabae98b336286b 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7       197.374433   
0xe095efc08fa11ab0891e0682a3a71ccf972eb6f7 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7         1.364660   
0xe0527e66ed8c7e372007de0d65a5eb45cac53e07 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7       264.641319   

                                                                                       count  
trader                                     fromToken                                          
0xa8db7ed8f1e3f18d7edd5ed4d6a4edabcf9c43b0 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7   5295  
0x7b3dedf3b349b8a943c18372972d150a572d2018 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e   1228  
                                           0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664    545  
                                           0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7    228  
0xa85a4eef551e083b2350a20994fd5816583cae3b 0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e   1500  
                                           0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7   1183  
0xa7b43b7afd3309fe93bb9f17145ce97ccda4f65e 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7    751  
0x6e74b6274e242f54840f78c8fbabae98b336286b 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7      6  
0xe095efc08fa11ab0891e0682a3a71ccf972eb6f7 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7    646  
0xe0527e66ed8c7e372007de0d65a5eb45cac53e07 0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7      3

In [85]:
def get_adapter_stats(arbs_df=yakswap_arbs):
    return adapter_swaps_df.merge(arbs_df, how="inner", on="txHash") \
        .groupby(by="adapter") \
        .agg(count=("txHash", "count"), profit_usd_sum=("profit_usd", "sum"), profit_usd_mean=("profit_usd", "mean")) \
        .sort_values(by="profit_usd_sum", ascending=False)

def get_hop_stats(arbs_df=yakswap_arbs):
    hop_counted = adapter_swaps_df.groupby(by="txHash").agg(hopCount=("fromAmount", "count"))
    swaps = hop_counted.merge(arbs_df, how="inner", on="txHash")
    return swaps \
        .groupby(by="hopCount") \
        .agg(count=("txHash", "count"), profit_usd_sum=("profit_usd", "sum"), profit_usd_mean=("profit_usd", "mean")) \
        .sort_values(by="profit_usd_sum", ascending=False)

def get_amountfrom_stats(arbs_df=yakswap_arbs):
    return arbs_df \
        .groupby("fromAmount") \
        .agg(count=("txHash", "count"), profit_usd_sum=("profit_usd", "sum"), profit_usd_mean=("profit_usd", "mean")) \
        .sort_values(by="profit_usd_sum", ascending=False)


def get_adapter_stats_for_trader_and_asset(trader, asset):
    arbs_trader = yakswap_arbs.loc[
        (yakswap_arbs.trader == trader) &
        (yakswap_arbs.fromToken == asset)
    ][["txHash", "profit_usd"]]
    return get_adapter_stats(arbs_trader)

def get_hop_stats_for_trader_and_asset(trader, asset):
    arbs_trader = yakswap_arbs.loc[
        (yakswap_arbs.trader == trader) &
        (yakswap_arbs.fromToken == asset)
    ][["txHash", "profit_usd"]]
    return get_hop_stats(arbs_trader)


def get_amountfrom_stats_for_trader_and_asset(trader, asset):
    arbs_trader = yakswap_arbs.loc[
        (yakswap_arbs.trader == trader) &
        (yakswap_arbs.fromToken == asset)
    ]
    return get_amountfrom_stats(arbs_trader)

### By assets

#### USDC

In [86]:
usdc_arbs = yakswap_arbs.loc[yakswap_arbs.fromToken == USDC]

In [87]:
get_adapter_stats(usdc_arbs)

,count,profit_usd_sum,profit_usd_mean
adapter,,,
PlatypusYakAdapterV2,2001,41852.199839,20.915642
Curve3poolfAdapterV0,724,29072.308976,40.155123
CurveYUSDAdapter,471,12836.403071,27.253510
WoofiUSDCAdapter,1467,11990.591511,8.173546
GmxAdapterV0,419,11910.260066,28.425442
TraderJoeYakAdapterV0,621,6787.027687,10.929191
KyberAdapter,721,3526.045174,4.890493
CurveUSDCAdapterV0,59,2585.404984,43.820423
CurveAtricryptoAdapterV0,108,611.817890,5.664980


In [88]:
get_hop_stats(usdc_arbs).head(5)

,count,profit_usd_sum,profit_usd_mean
hopCount,,,
2,1534,54399.879814,35.462764
3,1219,4362.728455,3.578940
5,1,11.044341,11.044341
4,1,4.282226,4.282226
6,1,2.479722,2.479722


In [89]:
get_amountfrom_stats(usdc_arbs).head(5)

,count,profit_usd_sum,profit_usd_mean
fromAmount,,,
100000.0,256,15963.354639,62.356854
7500.0,562,15697.456240,27.931417
25000.0,233,14001.641994,60.092884
50000.0,29,2330.056818,80.346787
250000.0,15,2130.061382,142.004092


#### WAVAX

In [96]:
wavax_arbs = yakswap_arbs.loc[yakswap_arbs.fromToken == WAVAX]

In [97]:
get_adapter_stats(wavax_arbs)

,count,profit_usd_sum,profit_usd_mean
adapter,,,
TraderJoeYakAdapterV0,7677,142941.319299,18.619424
GmxAdapterV0,1409,85113.864113,60.407285
WoofiUSDCAdapter,6315,40647.750591,6.436698
PlatypusYakAdapterV2,3726,28312.377297,7.598598
CurveAtricryptoAdapterV0,598,27322.517900,45.689829
KyberAdapter,1613,24423.126641,15.141430
Curve3poolfAdapterV0,135,18885.237439,139.890648
WoofiAdapter,1000,16232.483318,16.232483
PangolinYakAdapterV0,585,7799.171587,13.331917


In [98]:
get_hop_stats(wavax_arbs)

,count,profit_usd_sum,profit_usd_mean
hopCount,,,
3,6639,98444.977413,14.828284
2,2191,49245.108988,22.476088
4,13,1045.033623,80.387202
1,162,0.000000,0.000000


In [99]:
get_amountfrom_stats(wavax_arbs).head(5)

,count,profit_usd_sum,profit_usd_mean
fromAmount,,,
1350.0,68,23980.891832,352.660174
200.0,1398,17491.740197,12.511974
600.0,225,11890.226010,52.845449
1000.0,46,11375.250233,247.288049
500.0,128,9048.559523,70.691871
